##سوال 5

In [1]:
!pip install ucimlrepo

In [3]:
import gdown

file_id = "1Gf36UKjDLthc_2uYJMTwvSLjpfJA2ZFv"
output_file = "UCIAir.dat"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output_file)

Downloading...
From: https://drive.google.com/uc?id=1Gf36UKjDLthc_2uYJMTwvSLjpfJA2ZFv
To: /content/UCIAir.dat
100%|██████████| 785k/785k [00:00<00:00, 8.65MB/s]


'UCIAir.dat'

In [4]:
import pandas as pd

data = pd.read_csv('/content/UCIAir.dat')

print(data)

                                                                                                            Date;Time;CO(GT);PT08.S1(CO);NMHC(GT);C6H6(GT);PT08.S2(NMHC);NOx(GT);PT08.S3(NOx);NO2(GT);PT08.S4(NO2);PT08.S5(O3);T;RH;AH;;
10/03/2004;18.00.00;2            6;1360;150;11                 9;1046;166;1056;113;1692;1268;13 6;48 9;0                                                7578;;                                                                          
10/03/2004;19.00.00;2;1292;112;9 4;955;103;1174;92;1559;972;13 3;47                             7;0  7255;;                                                NaN                                                                          
10/03/2004;20.00.00;2            2;1402;88;9                   0;939;131;1140;114;1555;1074;11  9;54 0;0                                                7502;;                                                                          
10/03/2004;21.00.00;2            2;1376;80;9                   2;948

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# --- Step 1: Load and Preprocess Data ---
# Fix delimiter issue
data = pd.read_csv(file_path, delimiter=';', low_memory=False)

# Drop unnecessary columns and handle missing values
data = data.dropna().reset_index(drop=True)

# Select relevant numerical columns
data = data.iloc[:, 2:-2]  # Remove date/time columns and empty last column

# Convert to numeric values
data = data.apply(pd.to_numeric, errors='coerce')

# Handle missing values (drop or fill)
data = data.dropna().reset_index(drop=True)

# Separate features and target
X = data.iloc[:, :-1].values  # All but last column as features
y = data.iloc[:, -1].values   # Last column as target

# Normalize data
scaler_X = MinMaxScaler(feature_range=(-1, 1))
scaler_y = MinMaxScaler(feature_range=(-1, 1))

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Step 2: Implement RBF Model ---
rbf_model = KernelRidge(kernel='rbf', alpha=0.01, gamma=1)
rbf_model.fit(X_train, y_train)
y_pred_rbf = rbf_model.predict(X_test)
mse_rbf = mean_squared_error(y_test, y_pred_rbf)

# --- Step 3: Implement Fuzzy Inference System (Sugeno) ---
# Define fuzzy variables
input_vars = [ctrl.Antecedent(np.linspace(-1, 1, 100), f'input_{i}') for i in range(X.shape[1])]
output_var = ctrl.Consequent(np.linspace(-1, 1, 100), 'output')

# Define membership functions for inputs and output
for var in input_vars:
    var.automf(3)  # Three membership functions: Low, Medium, High

output_var['low'] = fuzz.trimf(output_var.universe, [-1, -0.5, 0])
output_var['medium'] = fuzz.trimf(output_var.universe, [-0.5, 0, 0.5])
output_var['high'] = fuzz.trimf(output_var.universe, [0, 0.5, 1])

# Define fuzzy rules
rules = []
for var in input_vars:
    rules.append(ctrl.Rule(var['poor'], output_var['low']))
    rules.append(ctrl.Rule(var['average'], output_var['medium']))
    rules.append(ctrl.Rule(var['good'], output_var['high']))

# Create and simulate fuzzy control system
fuzzy_control = ctrl.ControlSystem(rules)
fuzzy_model = ctrl.ControlSystemSimulation(fuzzy_control)

# Generate predictions using fuzzy model
y_pred_fuzzy = []
for i in range(len(X_test)):
    for j, var in enumerate(input_vars):
        fuzzy_model.input[var.label] = X_test[i, j]
    fuzzy_model.compute()
    y_pred_fuzzy.append(fuzzy_model.output['output'])

y_pred_fuzzy = np.array(y_pred_fuzzy)
mse_fuzzy = mean_squared_error(y_test, y_pred_fuzzy)

# --- Step 4: Plot Results ---
plt.figure(figsize=(12, 5))
plt.plot(y_test, label='Actual', linestyle='-', color='blue')
plt.plot(y_pred_rbf, label='RBF Prediction', linestyle='--', color='red')
plt.plot(y_pred_fuzzy, label='Fuzzy Prediction', linestyle='-.', color='green')
plt.xlabel('Sample Index')
plt.ylabel('Output')
plt.legend()
plt.title('Comparison of RBF and Fuzzy Predictions')
plt.grid(True)
plt.show()

# Error comparison
print(f"RBF Model MSE: {mse_rbf:.4f}")
print(f"Fuzzy Model MSE: {mse_fuzzy:.4f}")
if mse_rbf < mse_fuzzy:
    print("RBF performed better.")
else:
    print("Fuzzy model performed better.")

# Plot error distribution
plt.figure(figsize=(10, 5))
sns.histplot(y_test - y_pred_rbf, bins=25, kde=True, color='red', label='RBF Error')
sns.histplot(y_test - y_pred_fuzzy, bins=25, kde=True, color='green', label='Fuzzy Error')
plt.legend()
plt.title('Error Distribution of RBF and Fuzzy Models')
plt.xlabel('Error')
plt.grid(True)
plt.show()



ModuleNotFoundError: No module named 'skfuzzy'